# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=root")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')


In [5]:
filepath = song_files[0]
print(filepath)

In [6]:
df = pd.read_json(filepath, typ='series')
df

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
song_data = list(song_data)
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data = [None if str(value) == 'nan' else value for value in list(artist_data)]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head(2)

artist       auth firstName gender  itemInSession lastName  length level  \
0   None  Logged In    Walter      M              0     Frye     NaN  free   
1   None  Logged In    Kaylee      F              0  Summers     NaN  free   

                            location method  page   registration  sessionId  \
0  San Francisco-Oakland-Hayward, CA    GET  Home  1540919166796         38   
1        Phoenix-Mesa-Scottsdale, AZ    GET  Home  1540344794796        139   

   song  status             ts  \
0  None     200  1541105830796   
1  None     200  1541106106796   

                                           userAgent  userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      39  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page == 'NextSong']
df.head(2)

artist       auth firstName gender  itemInSession lastName     length  \
2  Des'ree  Logged In    Kaylee      F              1  Summers  246.30812   
4  Mr Oizo  Logged In    Kaylee      F              3  Summers  144.03873   

  level                     location method      page   registration  \
2  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   
4  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   

   sessionId          song  status             ts  \
2        139  You Gotta Be     200  1541106106796   
4        139       Flat 55     200  1541106352796   

                                           userAgent  userId  
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head(2)

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
Name: ts, dtype: datetime64[ns]

In [16]:
# Get timestamp, hour, day, week, month, year, weekday
time_data = [
    df['ts'],
    t.dt.hour,
    t.dt.day,
    t.dt.week,
    t.dt.month,
    t.dt.year,
    t.dt.weekday
]
# Get only values and make it a list
time_data = [list(column.values) for column in time_data]

# Sanity check for data integrity
if not all(len(i) == len(time_data[0]) for i in time_data[1:]):
    print('Warning - timestamp and other datetime attributes are not the same size')

# Check time data
display(list(zip(*time_data))[:2])

column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']

[(1541106106796, 21, 1, 44, 11, 2018, 3),
 (1541106352796, 21, 1, 44, 11, 2018, 3)]

In [17]:
time_df = pd.DataFrame(time_data).transpose()
time_df.columns = column_labels
time_df.head(2)

timestamp  hour  day  week  month  year  weekday
0  1541106106796    21    1    44     11  2018        3
1  1541106352796    21    1    44     11  2018        3

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(20)

userId firstName lastName gender level
2        8    Kaylee  Summers      F  free
4        8    Kaylee  Summers      F  free
5        8    Kaylee  Summers      F  free
6        8    Kaylee  Summers      F  free
7        8    Kaylee  Summers      F  free
8        8    Kaylee  Summers      F  free
9        8    Kaylee  Summers      F  free
10      10    Sylvie     Cruz      F  free
12      26      Ryan    Smith      M  free
13      26      Ryan    Smith      M  free
14     101    Jayden      Fox      M  free

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

##### User's Note: Inserting all the artists and songs

* We can use these to effectively test our SELECT query for songplays' artist and song IDs if needed - loads all artists and songs data

In [21]:
song_files = get_files('data/song_data')
filepaths = [file for file in song_files if '.ipynb' not in file]
songs_data = pd.DataFrame()
for file in filepaths:
    file_data = pd.read_json(file, typ='series')
    songs_data = songs_data.append(file_data, ignore_index=True)

In [22]:
# Convert these columns to integer
int_columns = [column for column in ['num_songs', 'year'] if column in songs_data.columns and songs_data[column].dtype != 'int64']
for column in int_columns:
    songs_data[column] = songs_data[column].astype('int64')
    
songs_data.head(2)

artist_id artist_latitude  artist_location artist_longitude  \
0  ARD7TVE1187B99BFB1            None  California - LA             None   
1  ARMJAGH1187FB546F3         35.1497      Memphis, TN         -90.0489   

    artist_name   duration  num_songs             song_id             title  \
0        Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   
1  The Box Tops  148.03546          1  SOCIWDW12A8C13D406         Soul Deep   

   year  
0     0  
1  1969

In [23]:
for idx, row in songs_data.iterrows():
    # Insert artists
    artist_data = list(row[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values)
    artist_data = [None if str(value) == 'nan' else value for value in list(artist_data)]
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
    
    # Insert songs
    song_data = row[['song_id', 'title', 'artist_id', 'year', 'duration']].values
    song_data = list(song_data)
    cur.execute(song_table_insert, song_data)
    conn.commit()


#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    songplay_data = [row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'],row['location'], row['userAgent']]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.